In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [80]:
folder = "../dataset/"
names=['zipcode','date','category','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
basic_stats = pd.io.parsers.read_table(folder + 'basic_stats000',sep='\t', names=names, parse_dates=["date"])

#payment_distribution 
#expenditure-time_curve
#demographic distribution

folder = "../dataset/"
names=['zipcode','date','category','zipcodecustomer','merchant','card', 'payment', 'avg', 'max', 'min', 'std']
customer_zipcodes = pd.io.parsers.read_table(folder + 'customer_zipcodes000',sep='\t', names=names, parse_dates=["date"])


basic_stats["amount"] = basic_stats["payment"]* basic_stats["avg"]
basic_stats["avgpaybymerch"] = basic_stats["payment"]/ basic_stats["merchant"]
basic_stats["amountbymerch"] = basic_stats["amount"]/ basic_stats["merchant"]
basic_stats["avgReppays"] = basic_stats["payment"] / basic_stats["card"]
basic_stats["weekday"] = basic_stats["date"].map(lambda d: (d.weekday()))
basic_stats["day"] = basic_stats["date"].map(lambda d: ('{0:%d}-{0:%a}'.format(d)))

customer_zipcodes["amount"] = customer_zipcodes["payment"]* customer_zipcodes["avg"]
customer_zipcodes["avgpaybymerch"] = customer_zipcodes["payment"]/ customer_zipcodes["merchant"]
customer_zipcodes["amountbymerch"] = customer_zipcodes["amount"]/ customer_zipcodes["merchant"]
customer_zipcodes["avgReppays"] = customer_zipcodes["payment"] / customer_zipcodes["card"]
customer_zipcodes["weekday"] = customer_zipcodes["date"].map(lambda d: (d.weekday()))
customer_zipcodes["day"] = customer_zipcodes["date"].map(lambda d: ('{0:%d}-{0:%a}'.format(d)))

In [ ]:
#basic_stats["density"]=Distribución normal (mean=avg, standarddeviation=std, cortadas( min , max)
# np.random.normal(mu, sigma, 1000)
#https://oneau.wordpress.com/2011/02/28/simple-statistics-with-scipy/

In [40]:
customer_zipcodes["amount"].sum()

132976729.3

In [15]:
customer_zipcodesbycp=pd.pivot_table(customer_zipcodes, values='amount', index=['zipcodecustomer'], aggfunc=np.sum)
customer_zipcodesbycategory=pd.pivot_table(customer_zipcodes, values='amount', index=['category'], aggfunc=np.sum)

In [34]:
customer_zipcodes_know=customer_zipcodes[customer_zipcodes["zipcodecustomer"]!='unknown']
customer_zipcodes_unknown=customer_zipcodes[customer_zipcodes["zipcodecustomer"]=='unknown']
customer_zipcodes_08002=customer_zipcodes[customer_zipcodes["zipcodecustomer"]=='08002']
customer_zipcodes_to_08002=customer_zipcodes[customer_zipcodes["zipcode"]==8002]

In [38]:
customer_zipcodes_knowbycp=pd.pivot_table(customer_zipcodes_know, values='amount', index=['zipcodecustomer'], aggfunc=np.sum)
print customer_zipcodes_know["amount"].sum()
print customer_zipcodes_unknown["amount"].sum()
print customer_zipcodes_know["amount"].sum()+customer_zipcodes_unknown["amount"].sum()
print customer_zipcodes["amount"].sum()

27152729.1
105824000.2
132976729.3
132976729.3


In [54]:
zips_to_08002=pd.pivot_table(customer_zipcodes_to_08002, values='amount', index=['zipcodecustomer'], aggfunc=np.sum)
zip_to_zips=pd.pivot_table(customer_zipcodes_08002, values='amount', index=['zipcode'], aggfunc=np.sum)

In [56]:
print zips_to_08002
zip_to_zips

zipcodecustomer
00000         4169.20
08001         8665.11
08002        18140.79
08003        18413.67
08004        10191.93
08005        10623.28
08006         1581.56
08009          989.50
08010         2141.64
08011         3059.46
08012         3863.40
08013        11627.84
08014         5508.77
08015        11568.97
08016         1674.99
08017          584.97
08018         5254.30
08019         1595.38
08020         2656.62
08021         1130.54
08022          892.73
08023         1991.30
08024         8833.97
08025         9107.66
08026         6815.85
08027         7283.11
08028         7497.98
08029         4122.54
08030         4502.36
08031         3273.84
              ...    
08530          376.32
08570          267.30
08640          292.62
08720          449.33
08740          724.54
08750          588.92
08754          100.85
08760          215.22
08800         1123.95
08820         2221.00
08830         3031.44
08840          359.52
08860          434.20
08901          8

zipcode
8001      278.90
8002    18140.79
8003      974.79
Name: amount, dtype: float64

In [66]:
import pip
def install(package):
   pip.main(['install', package])
install('pandasql') 
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

You are using pip version 7.0.1, however version 7.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [102]:
zipcode=''

queryfrom="""

SELECT category,zipcodecustomer,sum(amount) total
FROM customer_zipcodes 
where zipcode=8172
group by category,zipcodecustomer 
order by sum(amount) desc,zipcodecustomer ,category

"""

queryto="""
SELECT category,zipcode,sum(amount) 
FROM customer_zipcodes 
where zipcodecustomer='08172' 
group by category ,zipcode
order by sum(amount) desc,zipcode, category 
"""

print pysqldf(queryfrom)
print pysqldf(queryto)

                 category zipcodecustomer      total
0              es_fashion         unknown  318336.85
1   es_barsandrestaurants         unknown  131551.63
2    es_wellnessandbeauty         unknown  117427.65
3                 es_home         unknown  105301.65
4               es_health         unknown   90316.94
5                 es_food         unknown   83192.10
6        es_otherservices         unknown   80896.23
7                 es_auto         unknown   73140.68
8                 es_food           08172   59402.83
9                 es_food           08173   34879.82
10             es_fashion           08172   34354.10
11             es_fashion           08173   29807.60
12              es_health           08172   17715.04
13  es_barsandrestaurants           08172   16166.31
14  es_barsandrestaurants           08173   14078.98
15   es_wellnessandbeauty           08172   13573.86
16             es_fashion           08195   13494.67
17   es_wellnessandbeauty           08173   10